<a href="https://colab.research.google.com/github/iignat/NNExamples/blob/main/%22OpenAI_CarRacing_v0_implementation%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://gym.openai.com/envs/CarRacing-v0/

# Подготовка окружения

In [ ]:
# Устанавливаем окружение из пакета Stable Baseline для OpenAI (для того, чтобы можно было использовать движок в колабе)
# Код, представленный в документации разработчика
!apt install swig cmake libopenmpi-dev zlib1g-dev # cmake - сборка - аналог GNU Make, но с другим синтаксисом, libopenmpi-dev распределённые вычисления в кластере для дебиан, zlib1g-dev - сжатие траффика
!pip install stable-baselines==2.5.1 box2d box2d-kengz # устанавливаем непосредственно сам stable baseline и движок box2d

# если убрать " > /dev/null 2>&1" то работа сервера будет выведена на экран, а этой командой перемещаем все в фоновый режим
!pip install gym pyvirtualdisplay > /dev/null 2>&1 # установка пакета для рендера видео
!pip install xvfbwrapper # установка обертки для пакета, который позволит воспроизводить видео в ячейке колаба  
!apt-get update # обновляем все пакеты до новейшей версии, чтобы все пакеты "подружились"
!sudo apt-get install xvfb # установка удаленного сервера, благодаря которому видео будет воспроизводиться в ячейке колаба
!apt-get install xdpyinfo # пакет для вывода процессов, происходящих на сервере

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
libopenmpi-dev is already the newest version (2.1.1-8).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubu

In [ ]:
import gym # здесь лежат все модели для 
from gym import logger as gymlogger # загрузим библиотеку для изменения параметра вывода ошибок
from gym.wrappers import Monitor # библиотека для обертки видео из хранилища в ячейку
gymlogger.set_level(40) # для правильного отображения ошибок
import numpy as np # библиотека массивов numpy
import random # библиотека для генерации случайных значений
import matplotlib # библиотека для визуализации процессов
import matplotlib.pyplot as plt # библиотека для построения графика
import glob # расширение для использования Unix обозначений при задании пути к файлу
import io # библиотека для работы с потоковыми данными
import base64 # расширение для преобразования в формат base64 (универсальный формат хранения сырых изображений в виде набора электрических сигналов)
from IPython.display import HTML # библиотека для кодирования в код HTML
import time # библиотека для расчета времени обучения

from IPython import display as ipythondisplay # для работы с "сырым" форматом (набор сигналов, а не пиксели)
from pyvirtualdisplay import Display # для создания окна дисплея

# Создаем объект дисплей, таким образом, симуляцию можно будет наблюдать в колабе
display = Display(visible=0, size=(1400, 900))
display.start()

%xmode Plain
%matplotlib inline

In [ ]:
  genom_len = 6400 #длина генома
  chromo_len = 4 #количество хромосом

# Вспомогательные функции

In [ ]:
def popul_sequest(popul,val,vivpercent=50.,reverse=True):
  """
  Сиквестирование популяции
  popul - популяция
  val - значения loss - функции
  vivpercent - граница сиквестирования (процент выживших)
  reverse - max/min
  """
  vivenum = int(vivpercent*len(popul)/100.)
  uni = sorted(zip(val,popul), key=lambda tup: tup[0],reverse=reverse)
  new_popul = [x[1] for x in uni]
  new_val = [x[0] for x in uni]
  return new_popul[:vivenum],new_val[:vivenum] 

def new_bot(bot1,bot2):
  """
  скрещивание двух ботов
  """
  bot = [ bot1[i] if np.random.normal(0,1) < 0.5 else bot2[i] for i in range(chromo_len)]
  return bot

def popul_reprod(popul,popul_size):
  """
  репродукция популяции до указанного размера
  popul - популяция
  popul_size - граница репродукции
  """
  while len(popul)<popul_size:
    new_bots = [new_bot(popul[i],popul[np.random.randint(0,len(popul))]) for i in range(len(popul))]
    popul += new_bots

  return popul[:popul_size]

def popul_mutation(popul,percent=50., mut = 0.25):
  """
  мутация популяции
  percent - процент мутантов
  mut - коэффициент мутации
  """

  global genom_len #длина генома
  global chromo_len #количество хромосом

  num = int(len(popul)*percent/100.)
  mutants_idx = np.random.randint(1,len(popul),num)
  for i in mutants_idx:
    k = np.random.randint(0,chromo_len)
    mutagen = list(np.random.uniform(-1,1,genom_len))
    popul[i][k] = [a*b for a,b in zip(popul[i][k],mutagen)]
  return popul

In [ ]:
def plot_scatt(x,y,c,save_file = None):
  fig,ax = plt.subplots(nrows=1,ncols=1)
  fig.set_size_inches(10,5)

  ax.scatter(x=x,y=y,c=c)
  ax.set_title('Что-то состоящее из трех классов')

  ax.grid(axis='both',b=True,which='major',linestyle='-')
  ax.grid(axis='both',b=True,which='minor',linestyle=':')
  ax.minorticks_on()
  if save_file is not None:
    plt.savefig(save_file)
  plt.show()

In [ ]:
def show_video():
  '''
  Функция, позволяющая записывать видео того,
  что происходит в окружении и отображать это в колабе
  '''
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Целевые функции

In [ ]:
#задаем оптимизируемую функцию для варианта 1
def target_func(a):
  global env
  result = 0
  observation = env.reset()
  while True:
    action = np.dot(a,observation[:80,:80,1].reshape(-1))
    observation,reward,done,_ = env.step(action)   
    result += reward
    if done: 
      break;
  env.close()
  return result

# Реализация алгоритма

In [ ]:
def genom_calculation(popul_size = 1000,v_min = 0,v_max = 1 , epochs = 1000, 
                      mutpercent = 50, mut = 0.25,vivpercent = 50., 
                      opt_func = target_func,eps = 0, verbose = 0, 
                      checkpoint_file_name = None):
  """
  Оптимизайия функции 3х переменных с использованием ген. алгоритма
  popul_size - размер популяции
  
  v_min - нижняя граница значений
  v_max - верхняя граница значений
  epochs - количество эпох
  """
  global genom_len #длина генома
  global chromo_len #количество хромосом

  try:
    if checkpoint_file_name is not None:
      popul = list(np.load(checkpoint_file_name))
    else:
      popul = list(1.*np.random.uniform(v_min,v_max,(popul_size,chromo_len,genom_len))) #инициализируем популяцию
  except:
    print('Can\'t load checkpoint data. Generating new population.')
    popul = list(1.*np.random.uniform(v_min,v_max,(popul_size,chromo_len,genom_len))) #инициализируем популяцию

  cur_time = time.time()
  for epoch in range(epochs):
    val = [opt_func(popul[i]) for i in range(popul_size)]
    #val = map(opt_func,popul)
    popul,val = popul_sequest(popul,val,vivpercent = vivpercent)

    if verbose == 1:
      print ('Epoch {}/{} Best val:{}'.format(epoch,epochs,val[0]))
  
    popul = popul_reprod(popul,popul_size)
    popul = popul_mutation(popul, percent = mutpercent, mut = mut )

    if checkpoint_file_name is not None:
      np.save(checkpoint_file_name, np.array(popul))
 
  print('Best val:{}, bot={}'.format(opt_func(popul[0]),popul[0]))
  t = time.time() - cur_time
  print ('Затрачено времени:',t) 

  return popul[0], t 


In [ ]:
# Запускаем среду
env = gym.make('CarRacing-v0')

In [ ]:
res, t = genom_calculation(epochs=1, popul_size = 10,verbose = 1,checkpoint_file_name='/content/drive/MyDrive/race_point.npy')

Track generation: 1160..1454 -> 294-tiles track
Track generation: 1299..1628 -> 329-tiles track
Track generation: 1163..1458 -> 295-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1202..1513 -> 311-tiles track
Track generation: 968..1220 -> 252-tiles track
Track generation: 1098..1376 -> 278-tiles track
Track generation: 1014..1277 -> 263-tiles track
Track generation: 1235..1548 -> 313-tiles track
Track generation: 1209..1515 -> 306-tiles track
Epoch 0/1 Best val:652.9880478087549
Track generation: 1364..1709 -> 345-tiles track
Best val:440.6976744186, bot=[[-2.12750747e-01  1.61981968e-03  7.81034489e-03 ...  1.89112974e-02
   7.79646560e-02  1.22785730e-02]
 [-1.04704936e-03  1.11739973e-02 -7.52788890e-04 ...  2.16064666e-03
  -3.22541907e-02  6.51338100e-02]
 [-2.05824411e-13 -8.53643990e-10  4.97363971e-12 ... -2.55299871e-11
   3.52334029e-13  7.16521274e-09]
 [ 4.46858345e-25 -6.16797559e-24 -1.68106752e-25 ...  4.71652308e-27
  -1.86095298e-21  2.4

In [ ]:
np.save('/content/drive/MyDrive/race_final.npy', np.array(res))

# Апробация модели

In [ ]:
env = wrap_env(env) # оборачиваем наше окружения для записи видео

In [ ]:
observation = env.reset() # обнуляем вектор наблюдения
result = 0
while True:
    env.render() # Рендер окружения
    action = np.dot(res,observation[:80,:80,1].reshape(-1))
    # В качестве действия будем подавать значения нашего лучшего бота
    observation,reward,done,_ = env.step(action) # остлеживаем все параметры для подсчета функции значения
    result += reward
    if done: 
      break;            
env.close()
print('Reward:',result )
show_video()

Track generation: 1103..1388 -> 285-tiles track
Reward: 678.1690140844943
